<a href="https://colab.research.google.com/github/Muhammad-Danish-Abbas/Agentic_AI/blob/main/Project_3_Function_Tool_Calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Project 3: LangChain Function/Tool Calling Calculator

In [4]:
!pip install langchain google-generativeai
!pip install --upgrade google-generativeai
!pip install python-dotenv # This line installs the missing package


import os
from dotenv import load_dotenv # Now this import should work
import google.generativeai as generativeai
from langchain.tools import tool

# ... (Rest of your code remains the same) ...

# Load environment variables from the .env file
load_dotenv()

# Fetch the Google Gemini API key from environment variable
GOOGLE_GEMINI_API_KEY = os.getenv("GOOGLE_GEMINI_API_KEY")

# Step 1: Initialize Google Gemini Model using the correct method
def gemini_model(query: str) -> str:
    """Use the Google Gemini API to generate text responses"""
    try:
        # Call the Google Gemini model with the chat method
        generativeai.configure(api_key=GOOGLE_GEMINI_API_KEY)

        # Use the `chat` method if it exists
        response = generativeai.chat(
            model="gemini-1",  # Adjust the model name if needed
            prompt=query
        )

        return response['text']

    except Exception as e:
        return f"Error: {e}"

# Step 2: Define the Calculator Tool
class Calculator:
    def calculate(self, expression: str) -> str:
        try:
            result = eval(expression, {"__builtins__": None}, {})
            return str(result)
        except Exception as e:
            return f"Error: {e}"

# Step 3: Create the LangChain Tool Wrapper using the @tool decorator
@tool
def calculator_tool(expression: str) -> str:
    """
    Perform basic arithmetic calculations.
    Input: A mathematical expression as a string (e.g., "2 + 2").
    Output: The result of the calculation as a string.
    """
    calc = Calculator()
    return calc.calculate(expression)

# Step 4: Handle user input and interact with the calculator tool
def handle_query(query: str):
    # Call the Gemini model for generating a response
    gemini_response = gemini_model(query)

    # Check if the query is related to a calculation
    if 'calculate' in query.lower():
        # Example of passing calculation queries to the calculator
        result = calculator_tool(query)
        return f"Gemini says: {gemini_response}\nCalculator says: {result}"

    return gemini_response

Response: Error: module 'google.generativeai' has no attribute 'chat'
Query: What is 25 multiplied by 4?
Response: Error: module 'google.generativeai' has no attribute 'chat'
----------------------------------------
Query: Now divide the result by 5.
Response: Error: module 'google.generativeai' has no attribute 'chat'
----------------------------------------
Query: Add 10 to that.
Response: Error: module 'google.generativeai' has no attribute 'chat'
----------------------------------------


In [5]:
query = "What is 15 divided by 3?"
response = handle_query(query)
print(f"Response: {response}")

Response: Error: module 'google.generativeai' has no attribute 'chat'


In [6]:
# Example with multiple queries
queries = [
    "What is 25 multiplied by 4?",
    "Now divide the result by 5.",
    "Add 10 to that."
]

for q in queries:
    print("Query:", q)
    print("Response:", handle_query(q))
    print("-" * 40)


Query: What is 25 multiplied by 4?
Response: Error: module 'google.generativeai' has no attribute 'chat'
----------------------------------------
Query: Now divide the result by 5.
Response: Error: module 'google.generativeai' has no attribute 'chat'
----------------------------------------
Query: Add 10 to that.
Response: Error: module 'google.generativeai' has no attribute 'chat'
----------------------------------------


#Function /tool Calling

In [7]:
!pip install -U -q "google-generativeai>=0.7.2"

In [8]:
from google.colab import userdata
import google.generativeai as genai

genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))

#Music Setup

In [20]:
def start_music():
    """Turn on the music system."""
    print("MUSICBOT: Music started.")

def set_music_volume(level: int):
    """Set the music volume. Music must be started for this to work."""
    print(f"MUSICBOT: Music volume set to {level}.")

def stop_music():
    """Stop the music."""
    print("MUSICBOT: Music stopped.")

music_controls = [start_music, set_music_volume, stop_music]
instruction = "You are a helpful music system bot. You can start music, adjust the volume, and stop music. Do not perform any other tasks."

model = genai.GenerativeModel(
    "models/gemini-1.5-flash", tools=music_controls, system_instruction=instruction
)

chat = model.start_chat()

from google.generativeai.types import content_types
from collections.abc import Iterable

def tool_config_from_mode(mode: str, fns: Iterable[str] = ()):
    """Create a tool config with the specified function calling mode."""
    return content_types.to_tool_config(
        {"function_calling_config": {"mode": mode, "allowed_function_names": fns}}
    )

# Mode set to "none"
tool_config = tool_config_from_mode("none")

response = chat.send_message(
    "Hi, I am John. Tell me about your work?", tool_config=tool_config
)
print(response.text)

# Mode set to "auto"
tool_config = tool_config_from_mode("auto")

response = chat.send_message("Start the music!", tool_config=tool_config)
print(response.parts[0])

chat.rewind()  # You are not actually calling the function, so remove this from the history.

# Available functions
available_fns = ["set_music_volume", "stop_music"]

tool_config = tool_config_from_mode("any", available_fns)

response = chat.send_message("Set the volume to 7!", tool_config=tool_config)
print(response.parts[0])

# Available functions with start_music
available_fns = ["start_music", "set_music_volume", "stop_music"]
tool_config = tool_config_from_mode("any", available_fns)

auto_chat = model.start_chat(enable_automatic_function_calling=True)
response = auto_chat.send_message("It's too quiet in here, start the music and set volume to 5, then stop it", tool_config=tool_config)
print(response.parts[0])


I'm a music system bot.  I can start playing music, adjust the volume higher or lower, and stop the music.

function_call {
  name: "start_music"
  args {
  }
}

function_call {
  name: "set_music_volume"
  args {
    fields {
      key: "level"
      value {
        number_value: 7
      }
    }
  }
}

MUSICBOT: Music started.
MUSICBOT: Music volume set to 5.0.
MUSICBOT: Music stopped.
text: "Done.\n"



In [21]:
tool_config = tool_config_from_mode("none")

response = chat.send_message(
    "Hi I am Danish,Tell me about your work?", tool_config=tool_config
)
print(response.text)

I'm a music system bot. I can start music playback, adjust the volume (louder or softer), and stop the music.



In [22]:
available_fns = ["start_music","set_music_volume", "stop_music"]
tool_config = tool_config_from_mode("any", available_fns)

auto_chat = model.start_chat(enable_automatic_function_calling=True)
respones = auto_chat.send_message("Start Music for Alif Aslam, set up music maximum and then stop", tool_config=tool_config)

MUSICBOT: Music started.
MUSICBOT: Music volume set to 10.0.
MUSICBOT: Music stopped.


#Lighting

In [23]:
def enable_lights():
    """Turn on the lighting system."""
    print("LIGHTBOT: Lights enabled.")


def set_light_color(rgb_hex: str):
    """Set the light color. Lights must be enabled for this to work."""
    print(f"LIGHTBOT: Lights set to {rgb_hex}.")


def stop_lights():
    """Stop flashing lights."""
    print("LIGHTBOT: Lights turned off.")


light_controls = [enable_lights, set_light_color, stop_lights]
instruction = "You are a helpful lighting system bot. You can turn lights on and off, and you can set the color. Do not perform any other tasks."

model = genai.GenerativeModel(
    "models/gemini-1.5-pro", tools=light_controls, system_instruction=instruction
)

chat = model.start_chat()

#Helper function for setting function_calling_config on tool_config

In [24]:
from google.generativeai.types import content_types
from collections.abc import Iterable


def tool_config_from_mode(mode: str, fns: Iterable[str] = ()):
    """Create a tool config with the specified function calling mode."""
    return content_types.to_tool_config(
        {"function_calling_config": {"mode": mode, "allowed_function_names": fns}}
    )

#Text-only mode: NONE"
means that I should not use any tools and should reply only with text, just like a normal chat.

In [25]:
tool_config = tool_config_from_mode("none")

response = chat.send_message(
    "Hi I am Danish,Tell me about your work?", tool_config=tool_config
)
print(response.text)

I can turn lights on and off, and I can set the color.



#AUTO mode:
I can decide whether to respond in text or use tools to give you the best answer.

In [26]:
tool_config = tool_config_from_mode("auto")

response = chat.send_message("Light this place up!", tool_config=tool_config)
print(response.parts[0])
chat.rewind();  # You are not actually calling the function, so remove this from the history.

function_call {
  name: "enable_lights"
  args {
  }
}



#Function-calling mode: ANY means the model must use a tool whenever it answers.


*  If allowed_function_names is given, the model can only use those tools
* If allowed_function_names is not given, the model can use any tool.


In [27]:
available_fns = ["set_light_color", "stop_lights"]

tool_config = tool_config_from_mode("any", available_fns)

response = chat.send_message("Make this place Skyblue!", tool_config=tool_config)
print(response.parts[0])

function_call {
  name: "set_light_color"
  args {
    fields {
      key: "rgb_hex"
      value {
        string_value: "87CEEB"
      }
    }
  }
}



#Automatic function calling

In [28]:
available_fns = ["enable_lights","set_light_color", "stop_lights"]
tool_config = tool_config_from_mode("any", available_fns)

auto_chat = model.start_chat(enable_automatic_function_calling=True)
respones = auto_chat.send_message("It's awful dark in here and Make color blue and than switch off the light", tool_config=tool_config)

LIGHTBOT: Lights enabled.
LIGHTBOT: Lights set to 0000FF.
LIGHTBOT: Lights turned off.


#brightness Lighting

In [29]:
from typing import Literal

def set_light_values(brightness: int, color_temp: Literal["daylight", "cool", "warm"]):
    """Set the brightness and color temperature of a room light. (mock API).

    Args:
        brightness: Light level from 0 to 100. Zero is off and 100 is full brightness
        color_temp: Color temperature of the light fixture, which can be `daylight`, `cool` or `warm`.

    Returns:
        A dictionary containing the set brightness and color temperature.
    """
    return {
        "brightness": brightness,
        "colorTemperature": color_temp
    }

model = genai.GenerativeModel(model_name='gemini-1.5-flash',
                              tools=[set_light_values])

chat = model.start_chat(enable_automatic_function_calling=True)
response = chat.send_message('Dim the lights so the room feels cozy and warm.')
print(response.text)

OK. I've dimmed the lights to 30% brightness and set the color temperature to warm.  Is there anything else?



#Parallel function calling

In [30]:
def power_disco_ball(power: bool) -> bool:
    """Powers the spinning disco ball."""
    print(f"Disco ball is {'spinning!' if power else 'stopped.'}")
    return True


def start_music(energetic: bool, loud: bool, bpm: int) -> str:
    """Play some music matching the specified parameters.

    Args:
      energetic: Whether the music is energetic or not.
      loud: Whether the music is loud or not.
      bpm: The beats per minute of the music.

    Returns: The name of the song being played.
    """
    print(f"Starting music! {energetic=} {loud=}, {bpm=}")
    return "Never gonna give you up."


def dim_lights(brightness: float) -> bool:
    """Dim the lights.

    Args:
      brightness: The brightness of the lights, 0.0 is off, 1.0 is full.
    """
    print(f"Lights are now set to {brightness:.0%}")
    return True

In [31]:
# Set the model up with tools.
house_fns = [power_disco_ball, start_music, dim_lights]

model = genai.GenerativeModel(model_name="gemini-1.5-flash", tools=house_fns)

# Call the API.
chat = model.start_chat()
response = chat.send_message("Turn this place into a party!")

# Print out each of the function calls requested from this single call.
for part in response.parts:
    if fn := part.function_call:
        args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
        print(f"{fn.name}({args})")

power_disco_ball(power=True)
start_music(loud=True, energetic=True, bpm=120.0)
dim_lights(brightness=0.5)


In [32]:
# Simulate the responses from the specified tools.
responses = {
    "power_disco_ball": True,
    "start_music": "Never gonna give you up.",
    "dim_lights": True,
}

# Build the response parts.
response_parts = [
    genai.protos.Part(function_response=genai.protos.FunctionResponse(name=fn, response={"result": val}))
    for fn, val in responses.items()
]

response = chat.send_message(response_parts)
print(response.text)

Party started!  Never Gonna Give You Up is playing, the disco ball is spinning, and the lights are dimmed to 50% brightness.

